# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import numpy as np

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bubaque,11.2833,-15.8333,27.24,77,100,6.84,GW,1720059180
1,1,mirnyy,62.5353,113.9611,17.88,42,95,3.18,RU,1720059182
2,2,san juan,-31.5375,-68.5364,9.82,34,21,2.47,AR,1720059183
3,3,puerto ayora,-0.7393,-90.3518,21.42,96,57,2.97,EC,1720059185
4,4,saipan,15.1355,145.7010,32.39,66,40,6.69,MP,1720059186


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600    
)

# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HEREA 

ideal_df=city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"]> 21)& (city_data_df["Wind Speed"]< 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
# YOUR CODE HERE
ideal_df = ideal_df.dropna(how='any',axis=0) 

# Display sample data
# YOUR CODE HERE
ideal_df.head ()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,monticello,45.3055,-93.7941,25.07,56,0,0.00,US,1720058677
50,50,lincoln,40.8000,-96.6670,25.07,72,0,3.09,US,1720059264
59,59,bethel,41.3712,-73.4140,21.06,71,0,0.45,US,1720059278
107,107,hami,42.8000,93.4500,26.99,28,0,2.98,CN,1720059147
116,116,kill devil hills,36.0307,-75.6760,24.43,74,0,2.57,US,1720059137


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
ideal_reduced = ideal_df[['Country', 'Lat','Lng','Humidity']]
ideal_reduced.insert(4, "Hotel name", "") 
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
ideal_reduced.head ()
# YOUR CODE HERE

,Country,Lat,Lng,Humidity,Hotel name
43,US,45.3055,-93.7941,56,
50,US,40.8000,-96.6670,72,
59,US,41.3712,-73.4140,71,
107,CN,42.8000,93.4500,28,
116,US,36.0307,-75.6760,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
 "filter":filter,
    "limit":limit,
    "bias":bias,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


Starting hotel search
bijelo polje - nearest hotel: Hotel Franca
shiprock - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
hamilton - nearest hotel: North Vista Manor
waycross - nearest hotel: Holiday Inn Express & Suites
jamestown - nearest hotel: DoubleTree Jamestown
bonne terre - nearest hotel: Bonneville Motel
laguna - nearest hotel: Holiday Inn Express & Suites
cristuru secuiesc - nearest hotel: Kúria Vendégház
badger - nearest hotel: Clarion Hotel & Suites
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
koryazhma - nearest hotel: Виконда
sparti - nearest hotel: Maniatis
revda - nearest hotel: Металлург
mermoz boabab - nearest hotel: La maison blanche
mount pleasant - nearest hotel: Old Village Post House
nova vicosa - nearest hotel: Pousada Pontal da Barra
rio de janeiro - nearest hotel: Ibis Rio porto atlantico
qorveh - nearest hotel: No hotel found
vaison-la-romaine - nearest hotel: Hostellerie le Beffro

,City,Country,Lat,Lng,Humidity,Hotel Name
13,bijelo polje,ME,43.0337,19.7470,51,Hotel Franca
39,shiprock,US,36.7855,-108.6870,14,No hotel found
58,port elizabeth,ZA,-33.9180,25.5701,60,Waterford Hotel
82,hamilton,US,39.1834,-84.5333,82,North Vista Manor
109,waycross,US,31.2136,-82.3540,94,Holiday Inn Express & Suites
125,jamestown,US,42.0970,-79.2353,88,DoubleTree Jamestown
193,bonne terre,US,37.9231,-90.5554,89,Bonneville Motel
202,laguna,US,38.4210,-121.4238,53,Holiday Inn Express & Suites
215,cristuru secuiesc,RO,46.2833,25.0333,60,Kúria Vendégház
219,badger,US,64.8000,-147.5333,42,Clarion Hotel & Suites


In [6]:
# Set parameters to search for a hotel
radius = # YOUR CODE HERE
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bijelo polje - nearest hotel: Hotel Franca
shiprock - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
hamilton - nearest hotel: North Vista Manor
waycross - nearest hotel: Holiday Inn Express & Suites
jamestown - nearest hotel: DoubleTree Jamestown
bonne terre - nearest hotel: Bonneville Motel
laguna - nearest hotel: Holiday Inn Express & Suites
cristuru secuiesc - nearest hotel: Kúria Vendégház
badger - nearest hotel: Clarion Hotel & Suites
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
koryazhma - nearest hotel: Виконда
sparti - nearest hotel: Maniatis
revda - nearest hotel: Металлург
mermoz boabab - nearest hotel: La maison blanche
mount pleasant - nearest hotel: Old Village Post House
nova vicosa - nearest hotel: Pousada Pontal da Barra
rio de janeiro - nearest hotel: Ibis Rio porto atlantico
qorveh - nearest hotel: No hotel found
vaison-la-romaine - nearest hotel: Hostellerie le Beffro

,City,Country,Lat,Lng,Humidity,Hotel Name
13,bijelo polje,ME,43.0337,19.7470,51,Hotel Franca
39,shiprock,US,36.7855,-108.6870,14,No hotel found
58,port elizabeth,ZA,-33.9180,25.5701,60,Waterford Hotel
82,hamilton,US,39.1834,-84.5333,82,North Vista Manor
109,waycross,US,31.2136,-82.3540,94,Holiday Inn Express & Suites
125,jamestown,US,42.0970,-79.2353,88,DoubleTree Jamestown
193,bonne terre,US,37.9231,-90.5554,89,Bonneville Motel
202,laguna,US,38.4210,-121.4238,53,Holiday Inn Express & Suites
215,cristuru secuiesc,RO,46.2833,25.0333,60,Kúria Vendégház
219,badger,US,64.8000,-147.5333,42,Clarion Hotel & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)